In [ ]:
import sys
import os
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import math

sys.path.append("..")
from src.nce.cd_cnce import CdCnceCrit
from src.nce.cd_mh_cnce import CdMHCnceCrit
from src.nce.per_cnce import PersistentCondNceCrit
from src.nce.per_cnce_batch import PersistentCondNceCritBatch
from src.nce.per_cnce_batch_eps import PersistentCondNceCritBatchEps
from src.cd.rbm_crit import RbmCrit
from src.cd.cd_gibbs import CdGibbsCrit
from src.cd.pers_cd_gibbs import PersistentCdGibbsCrit
from src.cd.pers_cd_gibbs_batch import PersistentCdGibbsCritBatch

from src.noise_distr.rbm_noise_distr import RbmNoiseDistr, ConditionalRbmNoiseDistr
from src.noise_distr.bernoulli import MultivariateBernoulli
from src.noise_distr.conditional_bernoulli import ConditionalMultivariateBernoulli, SpatialConditionalMultivariateBernoulli
from src.noise_distr.uniform_sphere import UniformSphere
from src.noise_distr.conditional_sphere import ConditionalSphere

from src.models.rbm.rbm import Rbm
from src.models.ebm.two_layer_ebm import Ebm
from src.data.mnist import MnistDataset

from src.training.model_training import train_model
from src.training.training_utils import no_stopping

from src.experiments.mnist_exp_utils import initialise_ebm_params, initialise_rbm_params

%load_ext autoreload
%autoreload 2

## EXPERIMENTS

In [ ]:
# Data specs
img_dim = 28
num_dims = img_dim**2

# Model specs
num_hidden = 1000

# Training specs
num_neg_samples = 5
lr = 0.1
batch_size = 64
num_epochs = 10

In [ ]:
# Check accuracy of model
def rbm_acc(rbm, data_loader, k=100):
    acc = 0
    for i, (y, idx) in enumerate(data_loader, 0):
        _, y_pred = rbm.sample(y, k=k)
        acc += (y_pred == y).type(torch.float).mean(dim=-1).sum()
        
    return acc / len(data_loader.dataset)

def placeholder_metric(model):
    return model.weights.mean()

In [ ]:
# Configurations to consider in experiments 

config_conditional_multi = {
    "criterion": CdCnceCrit,
    "label": "cd_cnce",
    "estimate_part_fn": False,
    "conditional_noise_distr": True,
    "mcmc_steps": 1,
}

config_per_cnce = {
    "criterion": PersistentCondNceCrit,
    "label": "per_cnce",
    "estimate_part_fn": False,
    "conditional_noise_distr": True,
    "mcmc_steps": None,
}

config_per_cnce_batch = {
    "criterion": PersistentCondNceCritBatch,
    "label": "per_cnce_batch",
    "estimate_part_fn": False,
    "conditional_noise_distr": True,
    "mcmc_steps": None,
}


config_per_cnce_batch_eps = {
    "criterion": PersistentCondNceCritBatchEps,
    "label": "per_cnce_batch",
    "estimate_part_fn": False,
    "conditional_noise_distr": True,
    "mcmc_steps": None,
}


config_cd_mh = {
    "criterion": CdMHCnceCrit,
    "label": "cd_mh",
    "estimate_part_fn": False,
    "conditional_noise_distr": True,
    "mcmc_steps": 1,
}


config_rbm_crit = {
    "criterion": CdGibbsCrit,
    "label": "cd_gibbs",
    "estimate_part_fn": False,
    "conditional_noise_distr": True,
    "mcmc_steps": 1,
}


config_cd_gibbs = {
    "criterion": RbmCrit,
    "label": "cd_gibbs",
    "estimate_part_fn": False,
    "conditional_noise_distr": True,
    "mcmc_steps": 1,
}


config_pers_gibbs = {
    "criterion": PersistentCdGibbsCrit,
    "label": "pers_gibbs",
    "estimate_part_fn": False,
    "conditional_noise_distr": True,
    "mcmc_steps": None,
}


config_pers_gibbs_batch = {
    "criterion": PersistentCdGibbsCritBatch,
    "label": "pers_gibbs_batch",
    "estimate_part_fn": False,
    "conditional_noise_distr": True,
    "mcmc_steps": None,
}



# TODO: Vi vill jämföra de två första (de övriga är bara med som tillfälliga referenser)
configs = [config_per_cnce_batch_eps]#config_per_cnce_batch, config_per_cnce_batch_eps]

In [ ]:
# Get data 
training_data = MnistDataset(train=True, root_dir="../src/data/datasets/")
rel_inds = (training_data.targets == 0) + (training_data.targets == 1) + (training_data.targets == 2) + (training_data.targets == 3) + (training_data.targets == 4)
training_data.y = training_data.y[rel_inds, :]
training_data.num_samples = training_data.y.shape[0]
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True)
                             
test_data = MnistDataset(train=False, root_dir="../src/data/datasets/")
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

In [ ]:
# Initialise model       
weights, vis_bias, hidden_bias = initialise_rbm_params(num_visible=num_dims, num_hidden=num_hidden)

In [ ]:
def get_cond_bern_params(y, eps=1e-1):
    num_dims = y.shape[-1]
    p_0, p_1 = torch.zeros((num_dims,)), torch.zeros((num_dims,))
    for i in range(num_dims):
                
        if y[y[:, i] < 0.5, i].size()[0] == 0:
            p_0[i] = 0.5
        else:
            p_0[i] = y[y[:, i] < 0.5, i].mean()
        
        if y[y[:, i] >= 0.5, i].size()[0] == 0:
            p_1[i] = 0.5
        else:
            p_1[i] = y[y[:, i] >= 0.5, i].mean()

    p_0[p_0 < eps] = eps
    p_1[p_1 > 1 - eps] = 1 - eps
    return p_0, p_1
        
p_0, p_1 = get_cond_bern_params(training_data.y)

In [ ]:
# Test noise distr. params

fig, ax = plt.subplots(2, 1)

num_samples = 16
y_true = training_data.y[:num_samples, :].clone()
y_true[y_true >= 0.5] = 1.0
y_true[y_true < 0.5] = 0.0

p_n_test = ConditionalMultivariateBernoulli(p_0, p_1) # 

y_sample = p_n_test.sample(torch.Size((num_samples, 1)), y_true.reshape(-1, 1, y_true.shape[-1]))


ax[0].imshow(np.transpose(torchvision.utils.make_grid(y_true.reshape(-1, 1, 28, 28), nrow=4).numpy(), (1, 2, 0)))
ax[1].imshow(np.transpose(torchvision.utils.make_grid(y_sample.reshape(-1, 1, 28, 28), nrow=4).numpy(), (1, 2, 0)))

In [ ]:
# Run experiments

for config in configs:
    
    # Make sure that these are "reinitialised"
    p_m, p_n, criterion = None, None, None

    p_m = Rbm(weights=weights.clone(), vis_bias=vis_bias.clone(), hidden_bias=hidden_bias.clone())

    
    if config["conditional_noise_distr"]:
        p_n = RbmNoiseDistr(p_m) #ConditionalMultivariateBernoulli(p_0, p_1)
    else:
        p_n = None
   
    if config["mcmc_steps"] is not None:
        criterion = config["criterion"](p_m, p_n, num_neg_samples, config["mcmc_steps"])
    else:
        criterion = config["criterion"](p_m, p_n, num_neg_samples)

    save_dir = None
    _ = train_model(criterion, placeholder_metric, train_loader, save_dir, num_epochs=num_epochs,
                    decaying_lr=True, weight_decay=1e-2, stopping_condition=no_stopping)
    
    # Save model
    torch.save(p_m.state_dict(), "res/params_rbm_" + config["label"])
    
    # Check test accuracy of model
    #acc = rbm_acc(p_m, test_loader)
    #print("Model accuracy: {}".format(acc))
    


In [ ]:
# Visualise results

num_samples = 8
fig, ax = plt.subplots(len(configs), 2)


if ax.ndim == 1:
    ax = ax.reshape(1, -1)

p_m = Rbm(weights=weights.clone(), vis_bias=vis_bias.clone(), hidden_bias=hidden_bias.clone())

for i, config in enumerate(configs):
    p_m.load_state_dict(torch.load("res/params_rbm_" + config["label"]))
    
    y_true = training_data.y[:num_samples, :] #  torch.distributions.Bernoulli(0.5).sample((num_samples, num_dims)) #
    y_true_pred_prob, y_true_pred = p_m.sample(y_true.clone(), k=10000)
    
    y_noise = torch.distributions.Bernoulli(0.5).sample((num_samples, num_hidden))  
    y_pred_prob, y_pred = p_m.sample_from_hidden(y_noise, k=10000) 
    
    ax[i, 0].imshow(np.transpose(torchvision.utils.make_grid(y_true_pred_prob.reshape(-1, 1, 28, 28), nrow=4).numpy(), (1, 2, 0)))
    ax[i, 0].set_title(config["label"] + " predicted proabilities")
    
    ax[i, 1].imshow(np.transpose(torchvision.utils.make_grid(y_pred_prob.reshape(-1, 1, 28, 28), nrow=4).numpy(), (1, 2, 0)))
    ax[i, 1].set_title(config["label"] + " samples")
    
plt.show()